In [1]:
%matplotlib inline
%reload_ext autoreload

In [2]:
import matplotlib.pyplot as plt
import numpy as np
import xarray as xr
import seaborn as sns
import logging

import logging
import sys

def setup_logging(epoch_key, date_format='%d-%b-%y %H:%M:%S', format='%(asctime)s %(message)s'):
    animal, day, epoch = epoch_key
    log_filename = f"{animal}_{day:02d}_{epoch:02d}_test_speed_gpu.log"

    logger = logging.getLogger()
    logger.setLevel(logging.INFO)
    formatter = logging.Formatter(fmt=format, datefmt=date_format)

    stdout_handler = logging.StreamHandler(sys.stdout)
    stdout_handler.setLevel(logging.INFO)
    stdout_handler.setFormatter(formatter)

    file_handler = logging.FileHandler(log_filename)
    file_handler.setLevel(logging.INFO)
    file_handler.setFormatter(formatter)

    logger.addHandler(file_handler)
    logger.addHandler(stdout_handler)

sns.set_context("talk")

In [3]:
epoch_key = 'Jaq', 3, 12

In [4]:
setup_logging(epoch_key)

In [5]:
from src.load_data import load_data

logging.info('GPU likelihood')
data = load_data(epoch_key,
                 position_to_linearize=['nose_x', 'nose_y'],
                 max_distance_from_well=5,
                 min_distance_traveled=30)

19-Sep-21 16:18:33 Loading position info...
19-Sep-21 16:19:07 Loading multiunit...
19-Sep-21 16:19:29 Loading spikes...
19-Sep-21 16:20:44 Finding ripple times...


#### GPU

In [6]:
from replay_trajectory_classification import ClusterlessClassifier
from src.parameters import WTRACK_EDGE_ORDER, WTRACK_EDGE_SPACING
import pprint

continuous_transition_types = (
    [['random_walk', 'uniform'],
     ['uniform',     'uniform']])


clusterless_algorithm = 'multiunit_likelihood_gpu'
clusterless_algorithm_params = {
    'mark_std': 20.0,
    'position_std': 8.0,
}

classifier_parameters = {
    'movement_var': 6.0,
    'replay_speed': 1,
    'place_bin_size': 2.5,
    'continuous_transition_types': continuous_transition_types,
    'discrete_transition_diag': 0.968,
    'clusterless_algorithm': clusterless_algorithm,
    'clusterless_algorithm_params': clusterless_algorithm_params
}

logging.info(pprint.pprint(classifier_parameters))

In [ ]:
state_names = ['Continuous', 'Fragmented']

classifier = ClusterlessClassifier(**classifier_parameters)
classifier.fit(
    position=data["position_info"].linear_position,
    multiunits=data["multiunits"],
    track_graph=data["track_graph"],
    edge_order=WTRACK_EDGE_ORDER,
    edge_spacing=WTRACK_EDGE_SPACING,
)

results_gpu_likelihood = classifier.predict(
    data["multiunits"],
    time=data["position_info"].index / np.timedelta64(1, "s"),
    state_names=state_names
)
logging.info('Done...')

19-Sep-21 16:21:16 GPU likelihood
19-Sep-21 16:21:17 Fitting initial conditions...
19-Sep-21 16:21:17 Fitting state transition...
19-Sep-21 16:21:17 Fitting multiunits...
19-Sep-21 16:21:17 init
19-Sep-21 16:21:19 add pending dealloc: cuMemFree_v2 636 bytes
19-Sep-21 16:21:19 add pending dealloc: cuMemFree_v2 1794852 bytes
19-Sep-21 16:21:19 add pending dealloc: cuMemFree_v2 4 bytes
19-Sep-21 16:21:19 add pending dealloc: cuMemFree_v2 636 bytes
19-Sep-21 16:21:19 add pending dealloc: cuMemFree_v2 636 bytes
19-Sep-21 16:21:19 add pending dealloc: cuMemFree_v2 158992 bytes
19-Sep-21 16:21:19 add pending dealloc: cuMemFree_v2 4 bytes
19-Sep-21 16:21:19 add pending dealloc: cuMemFree_v2 636 bytes
19-Sep-21 16:21:19 add pending dealloc: cuMemFree_v2 636 bytes
19-Sep-21 16:21:19 add pending dealloc: cuMemFree_v2 353964 bytes
19-Sep-21 16:21:19 add pending dealloc: cuMemFree_v2 4 bytes
19-Sep-21 16:21:19 dealloc: cuMemFree_v2 636 bytes
19-Sep-21 16:21:19 dealloc: cuMemFree_v2 1794852 bytes
19

## GPU Likelihood + State Space

In [ ]:
logging.info('GPU likelihood + State Space')
state_names = ['Continuous', 'Fragmented']

classifier = ClusterlessClassifier(**classifier_parameters)
classifier.fit(
    position=data["position_info"].linear_position,
    multiunits=data["multiunits"],
    track_graph=data["track_graph"],
    edge_order=WTRACK_EDGE_ORDER,
    edge_spacing=WTRACK_EDGE_SPACING,
)

results_gpu_likelihood = classifier.predict(
    data["multiunits"],
    time=data["position_info"].index / np.timedelta64(1, "s"),
    state_names=state_names
    use_gpu=True
)

In [ ]:
logging.info('Done...')